In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import pandas as pd
from tqdm import tqdm

In [11]:
inPath = "/shared/3/projects/benlitterer/podcastData/hostIdentification/trainTestVal2_5/descTrainLabelled.csv"
df = pd.read_csv(inPath)

In [12]:
df.shape

(30158, 15)

In [5]:
df = df.dropna(subset=["tag"])

In [6]:
df[["left", "ent", "right"]] = df[["left", "ent", "right"]].fillna("")

In [7]:
df["fullSnip"] = df["left"] + df["ent"] + df["right"]

In [8]:
deviceNum = 1
device = torch.device("cuda:" + str(deviceNum) if torch.cuda.is_available() else "cpu")

In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small").to(device)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

[2024-02-08 18:07:55,785] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [10]:
outPred = []
for snip in tqdm(df["fullSnip"]):  
    prefix = "Output the full name of the podcast hosts in the following sequence of text, and if there is no host output NONE instead: "  
    inputs = tokenizer(f"{prefix} {snip}", return_tensors="pt").to(device)
    outputs = model.generate(**inputs)
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    outPred.append(decoded) 
    #print(snip) 
    #print(decoded) 
    #print("----------------------------") 

  0%|          | 0/298 [00:00<?, ?it/s]/opt/anaconda/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 298/298 [00:33<00:00,  8.98it/s]


In [13]:
outPred[0:10]

[['NONE'],
 ['NONE instead:'],
 ['Geneviève Kyle-Lefebvre'],
 ['Robert Waterbury'],
 ['Nicole Scherzinger'],
 ['Richard Grannon'],
 ['Greg Garcia'],
 ['Christian faith based Ecumenical poems'],
 ['Mark Hawkinson'],
 ['Tiny Leaps, Big Changes']]

In [15]:
outCol = pd.DataFrame(outPred, columns=["FLANT5oneShot"])

In [16]:
outCol.to_csv("/shared/3/projects/benlitterer/podcastData/hostIdentification/FLANTT5/earlyExperiments/descTrainTest.csv", sep=",")